In [1]:
import db_conn
import pandas as pd
import re

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 약물
추출된 전체 약물 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_drugs():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(d.tt_drug)) AS t_d
    FROM
    (SELECT id, tt_drug
    FROM article_table_sentences_m M,
        UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
    WHERE (tt_drug='') is false) as d;
    """)
    return cur.fetchall()

In [8]:
drugs = pd.DataFrame(get_total_drugs())

In [9]:
drugs.to_csv('./drugs.csv', sep=',')

In [ ]:
drugs.columns=['name']

In [ ]:
drugs['refined'] = ''
drugs.head()

#### 전처리 전 총 약물의 수는 2323개 입니다

In [ ]:
drugs.shape

### 2. 전처리

#### A. 약물을 포함하고 있는 구절이 추출 된 경우, 약물 외적인 단어들을 제거합니다

In [ ]:
replace_with_space = ['-long-acting', '-treated', '-producing', '-high', '-involved-field', '-based', '-studies', '-non-susceptible',\
                      '-susceptible', '-dependent', '-inducible', '-independent', '-eligible', '-rescue', '-sensitive',\
                      'infarction-', '11c]-', '[99mtc]', '-avid', '-subcutaneous', '-coated', '-activatable', '-total', '-corrected', \
                      '-predicted', '-gated', '-redo-surgery', '-group', '-assigned', '-allocated', '-reduced', '-combination', \
                      '-standard', '-associated', '-ischemic', '-resistant', '-naive', '-sparing', 'concentration', \
                      'low-molecular-weight-', '-extended', '-prev', '-meter', '-stimulated', 'subtotal', '-labeled', '-matching',\
                      '-resistance;', '-maintenance', '-rate', '-tolerance', '-facilitated', '-replacement', '-to-renin', '-gits', \
                      '-right', '-or-2-h', '-treatment', '-boosting', '-resistance', '-continuation', '-rich', '-insensitive',\
                      '-impregnated', '-dispensation', '-pemetrexed', '-correctable', '-carry-on', '-oral', '-intravenous', '-bare-metal', '-dose',\
                      '-equivalent', '-induced', '-positive', '-related', '-stimulating', '-confirmed', '-controlled', '(',\
                      '-releasing', '-reuptake', '-eluting', '-containing', '-binding', 'vaccines', 'vaccine', 'change',\
                      # 약물 이름이 아닌 경우
                      'food-borne', 'fast-food', 'food-coping', 'grass-pea-food', 'food-systems', 'food-bolus', 'food-specific', \
                      'food-frequency', 'non-food', 'blood plasma', 'medical air', 'tree pollen', 'grass pollen'
                     ]

In [ ]:
# .....'-the', 'on-', 
_replace_with_space = ['-receptor'
                      'post-','-dose-dense', '–placebo', \
                      '-inhibiting', '-containing', '-related', '-like', '-boosted', '-binding', '8 × ', '-only', \
                      '-pulse', '-positive', '-confirmed', '-controlled', '-releasing', '-lowering', 'pre-', '≥110', \
                      '-independent', '≥5', 'secreting', 'mothers', 'bothersome', "mothers'", 'senegal']

# '-dependent', , 

In [ ]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        drugs.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        drugs.loc[ind]['refined'] = refined
        print(row['name'])
        print(drugs.loc[ind]['refined'])
    else:
        print(row['name'])
    print('===============================')

drugs = drugs.reset_index(drop=True)

#### 전처리와 약물이 아닌 레코드를 제거하면 2310개가 남습니다

In [ ]:
drugs.shape

#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

In [ ]:
non_drug_ad_rel = ['pseudo', 'placebo', 'negative', 'free', 'withdrawal', 'blocking', 'deficient', 'inhibiting']
# 'no-', 

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [ ]:
for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        drugs.drop(ind, inplace=True)
        print('===============================')

drugs = drugs.reset_index(drop=True)

In [ ]:
non-

#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 2290 남습니다

In [ ]:
drugs.shape

In [ ]:
drugs.loc[drugs.name.str.find('(')>=0]

In [ ]:
drugs[drugs.refined==''].loc[drugs.name.str.find(' ')>=-1][350:400]

# split :, =, —
# 'factor', 'guar gum'

# 'oxygenator', 'non-secretin', '<0.001', 'others', 'mothers', 'chemotherapy', '<3.2', 'daily+', '+late', '-refractory'
# '—1688'

In [ ]:
drugs[drugs.refined=='']

### D. 정제

In [ ]:
ends_with = ['.', ':', '-']
starts_with = ['-', '%']

### 선생님께 확인 요청을 드리고 싶은 부분
- 약물명 중에 '+'가 표기 되어있는 경우가 있는데, 이 경우 어떻게 약물명을 정제하면 좋을까요?

In [ ]:
drugs.loc[drugs.name.str.find('+')>=0]